In [114]:
import requests
from tqdm.auto import tqdm
import json
import pandas as pd
from datetime import datetime


### Ideias de classificação

 Com base na descrição da pra vericar se houve mortos ou não

 Tem também o parâmetro crash, da pra usar ele como alguma coisa na hora de classificar

 Da pra criar com base nos components uma classificação voltada para o texto em summary.



## 1. Aquisição e Pré-processamento de Dados

### Objetivo

* Nesta seção, detalhamos o processo de aquisição e pré-processamento dos dados de reclamações de veículos da NHTSA. O objetivo foi preparar os dados para treinamento de um modelo de classificação baseado em embeddings.

### Aquisição dos dados

* Os dados foram obtidos através da API fornecida do endereço `https://api.nhtsa.gov`

* Com o uso da biblioteca requests, do Python, foi possível adquirir as informações acerca das reclamações de veículos do NHTSA

In [93]:
def get_all_model_years() -> dict:
    """
    Obtem todos o ano de todos os modelos dispoíveis.
    """
    url = "https://api.nhtsa.gov/products/vehicle/modelYears?issueType=c"
    try:
        response = requests.get(url)
        results = response.json().get('results')
        if results:
            return [year['modelYear'] for year in results]
    except Exception as e:
        print(f"Ocorreu um erro: {e}")
    return None

In [94]:
def get_all_makes_model_year(list_model_year: list) -> dict:
    """
    A partir da lista de anos dos modelos disponíveis, obtem a marca daquele modelo.
    """
    url = "https://api.nhtsa.gov/products/vehicle/makes?modelYear={model_year}&issueType=c"
    # Lista com os modelos e anos.
    make_modelyear = []
    for model_year in tqdm(list_model_year, desc="Capturando modelo e ano", colour="green"):
        response = requests.get(url.format(model_year=model_year))
        results = response.json().get('results')
        if results:
            make_modelyear.extend(results)
    return make_modelyear

In [95]:
def get_all_models_make_year(list_make_year: list[dict]) -> list[dict]:
    """
    Obtem uma lista de dicionários que contém a marca, o modelo e o ano para cada carro.
    """
    url = "https://api.nhtsa.gov/products/vehicle/models?modelYear={modelYear}&make={make}&issueType=c"
    # Lista com modelos, marcas e anos.
    model_make_year = []
    for item in tqdm(list_make_year, desc="Capturando marca, modelo e ano", colour="yellow"):
        response = requests.get(url.format(modelYear=item.get('modelYear'), make=item.get("make")))
        results = response.json().get('results')
        if results:
            model_make_year.extend(results)
    return model_make_year

In [ ]:
list_model_year = get_all_model_years()
list_make_year = get_all_makes_model_year(list_model_year)

In [ ]:
list_make_model_year = get_all_models_make_year(list_make_year)

In [ ]:
len(list_make_model_year)

In [ ]:
# Dropando duplicados
make_model_year = pd.DataFrame(list_make_model_year)
make_model_year.drop_duplicates(inplace=True)
make_model_year.dropna(inplace=True)
make_model_year = make_model_year.to_dict(orient='records')
make_model_year

In [ ]:
len(make_model_year)

In [110]:
json.dump(make_model_year, open("make_model_year.json", "w"), indent=4, ensure_ascii=False)

#### Obtendo todas as reclamações

In [124]:
def get_all_complaints(make_model_year: list[dict]) -> list[dict]:
    """
    Obtem uma lista de dicionários que contém as reclamações por modelo, marca e ano.
    """
    url = "https://api.nhtsa.gov/complaints/complaintsByVehicle?make={make}&model={model}&modelYear={modelYear}"
    # Lista das reclamações por modelo, marca e ano.
    complaints = []
    for item in tqdm(make_model_year, desc="Capturando as reclamações", colour="blue"):
        response = requests.get(url.format(make=item.get('make'), model=item.get('model'), modelYear=item.get('modelYear')))
        results = response.json().get('results')
        if results:
            for result in results:
                try:
                    date = result.get('dateComplaintFiled')
                    if date:
                        data_formatada = datetime.strptime(date, '%m/%d/%Y')
                        year = data_formatada.year
                        if year >= 2014 and year <= 2024:
                            complaints.extend(result)
                except Exception as e:
                    print(f"Erro ao processar o registro: {e}")
                    pass
    return complaints

In [ ]:
get_all_complaints(make_model_year)

In [ ]:
data = '08/29/2011'

# Converter a string para um objeto datetime
data_formatada = datetime.strptime(data, '%m/%d/%Y')

# Extrair o ano da data
ano = data_formatada.year
ano > 2014